In [6]:
import numpy as np
import sys
sys.path.append("../")

from model_select import (find_scores, auc_alc, random_sequence, 
                          convex_hull_sequence)

In [2]:
scores = find_scores("../store_dir/", methods=dict(random=random_sequence,
                                                   convex_hull=convex_hull_sequence))

165/165 [==============================] - 0s 292us/step
method=random, fraction=0.0, loss=3.336443802804658, acc=0.012121212121212121
165/165 [==============================] - 0s 245us/step
method=random, fraction=0.05, loss=6.068713257529518, acc=0.4060606062412262
165/165 [==============================] - 0s 259us/step
method=random, fraction=0.1, loss=5.238098271687826, acc=0.5151515153321353
165/165 [==============================] - 0s 279us/step
method=random, fraction=0.15, loss=1.919926439632069, acc=0.7515151522376321
165/165 [==============================] - 0s 410us/step
method=random, fraction=0.2, loss=1.1851755019390222, acc=0.8121212124824524
165/165 [==============================] - 0s 374us/step
method=random, fraction=0.25, loss=0.9950248026486599, acc=0.8848484852097251
165/165 [==============================] - 0s 403us/step
method=random, fraction=0.3, loss=1.0156840539339818, acc=0.8484848488460888
165/165 [==============================] - 0s 449us/step
meth

In [3]:
from bokeh.io import output_notebook, show, reset_output
from bokeh.plotting import figure, curdoc, ColumnDataSource
from bokeh.models import HoverTool, Range1d

output_notebook()

Loading BokehJS ...

In [4]:
def make_upper(score):
    upper = np.array(list(map(lambda fract: (fract[0], fract[1][1]),
                              sorted(score.items(), key=lambda x: x[0]))))
    return upper[:, 0], upper[:, 1]

def make_patch(upper_x, upper_y):
    lower_x, lower_y = (upper_x[np.arange(upper_x.shape[0] - 1, -1, -1)], 
                        np.zeros_like(upper_y))
    return np.hstack((upper_x, lower_x)), np.hstack((upper_y, lower_y))

def make_percent_list(fracts):
    return [f"{round(fract * 100, 2)}%" for fract in fracts]

## Just got a keyboard! Thanks to Rao :)

In [7]:
p = figure(plot_width=600, plot_height=450, tools="save")

hover_tool_renderers = []
colors = ["blue", "red", "green", "orange"]
for idx, (name, score) in enumerate(scores.items()):
    upper_x, upper_y = make_upper(score)
    patch_x, patch_y = make_patch(upper_x, upper_y)

    source = ColumnDataSource(data=dict(
        split=make_percent_list(upper_x), 
        accuracy=make_percent_list(upper_y), x=upper_x, 
        y=upper_y, name=[name] * upper_x.shape[0]
    ))

    auc_score = round(auc_alc(upper_y, 0.05), 4)
    legend_name = f"{name} ({auc_score})"

    color = colors[idx]
    method_area = p.patch(patch_x, patch_y, fill_alpha=0.4, fill_color=color,
                          line_alpha=0.4, muted_alpha=0.1, legend=legend_name)
    method_line = p.line("x", "y", source=source, line_alpha=0.4, 
                         line_color=color, muted_alpha=0.1, legend=legend_name)
    hover_tool_renderers.append(method_line)

hover_tool = HoverTool(
    tooltips=[("Split", "@split"), ("Accuracy", "@accuracy"), 
              ("Method", "@name")],
    renderers=hover_tool_renderers, 
)
p.add_tools(hover_tool)
p.xaxis.axis_label = "Fraction of Training Data"
p.x_range = Range1d(0, 1)
p.y_range = Range1d(0, 1)
p.yaxis.axis_label = "Test Accuracy"
p.legend.click_policy = "hide"
p.legend.location = "bottom_right"
show(p)